# Compilation


Just like qubit-based quantum computations can be decomposed into elementary one- and two-qubit gates, arbitrary continuous-variable operations can be decomposed into gates such as two-mode beamsplitters and single-mode rotations. In these exercises, you will learn how to perform such *compilation* using the tools in Strawberry Fields, including how to compile programs to run on Xanadu hardware. 

**Exercise 1.** Using a tool of your choice, generate a random 8 x 8 unitary matrix. (This is available directly in Strawberry Fields, but you can also use other libraries.

In [1]:
import numpy as np
import strawberryfields as sf
from strawberryfields import ops

U = sf.utils.random_interferometer(8)

2021-07-30 11:04:36.786267: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-07-30 11:04:36.786286: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


**Exercise 2.** Write a program that applies your unitary to 8 modes using the `Interferometer` operation, then measures the outcome in the Fock basis. Print out the execution of your program using `prog.print()`.

In [2]:
prog = sf.Program(8)

with prog.context as q:
    ops.Interferometer(U) | q
    ops.MeasureFock() | q

In [3]:
prog.print()

Interferometer([[-7.1626e-02+0.0966j -5.1517e-02+0.1693j -1.4009e-01-0.5638j
  -1.6298e-01-0.051j   1.5309e-01-0.5444j -2.0009e-01+0.2816j
  -5.3470e-04-0.254j  -2.8875e-01+0.0216j]
 [-2.4115e-01+0.0388j  4.0037e-01-0.1916j  3.3270e-02+0.0107j
   2.8802e-02+0.04j   -2.6493e-02-0.1613j -7.3141e-02-0.0345j
  -5.4438e-01+0.205j  -8.8481e-02-0.6002j]
 [-5.8923e-01+0.2309j  3.1642e-01-0.272j   4.0586e-02+0.0214j
  -1.3058e-01+0.0157j -2.6301e-01-0.035j  -9.7365e-03-0.1496j
   4.6540e-01-0.2995j  7.0252e-02+0.0431j]
 [-4.2811e-02-0.1581j -2.4550e-01+0.0502j -1.6712e-01+0.0588j
   4.4143e-01-0.2049j -1.0854e-01-0.1244j  3.2878e-01-0.4245j
   1.4041e-02-0.3545j -3.8573e-01-0.2279j]
 [ 7.4000e-02+0.419j  -9.3672e-02+0.1028j -4.2774e-01+0.3764j
  -1.5994e-01-0.3429j  2.9370e-01-0.2377j -2.8002e-02-0.0806j
   2.1805e-01+0.1659j  2.0117e-01-0.2573j]
 [ 1.4668e-01-0.1357j -9.5039e-03+0.0571j -2.4766e-01-0.0671j
   3.8473e-01+0.139j  -3.2841e-01-0.0106j -7.0697e-01-0.0793j
   2.2300e-01+0.0564j  1.5

From this description, it's very hard to tell how this will look when implemented in practice!

**Exercise 3.** [Compile](https://strawberryfields.readthedocs.io/en/stable/introduction/circuits.html#compilation) your program using the `fock` compiler, and print the output.

In [4]:
prog2 = prog.compile(compiler="fock")

In [5]:
prog2.print()

Rgate(0.4807) | (q[0])
BSgate(0.8199, 0) | (q[0], q[1])
Rgate(1.388) | (q[2])
BSgate(0.312, 0) | (q[2], q[3])
Rgate(2.823) | (q[1])
BSgate(1.088, 0) | (q[1], q[2])
Rgate(-0.6411) | (q[0])
BSgate(0.7177, 0) | (q[0], q[1])
Rgate(0.3256) | (q[4])
BSgate(0.2993, 0) | (q[4], q[5])
Rgate(1.877) | (q[3])
BSgate(1.223, 0) | (q[3], q[4])
Rgate(-1.496) | (q[2])
BSgate(1.533, 0) | (q[2], q[3])
Rgate(-2.696) | (q[1])
BSgate(0.8028, 0) | (q[1], q[2])
Rgate(-0.3495) | (q[0])
BSgate(1.084, 0) | (q[0], q[1])
Rgate(-1.498) | (q[6])
BSgate(0.8507, 0) | (q[6], q[7])
Rgate(-0.3996) | (q[5])
BSgate(1.043, 0) | (q[5], q[6])
Rgate(1.013) | (q[4])
BSgate(1.307, 0) | (q[4], q[5])
Rgate(2.449) | (q[3])
BSgate(1.216, 0) | (q[3], q[4])
Rgate(-2.14) | (q[2])
BSgate(0.9308, 0) | (q[2], q[3])
Rgate(-0.5946) | (q[1])
BSgate(0.9157, 0) | (q[1], q[2])
Rgate(-1.396) | (q[0])
BSgate(0.8857, 0) | (q[0], q[1])
Rgate(6.208) | (q[0])
Rgate(1.174) | (q[1])
Rgate(2.109) | (q[2])
Rgate(1.906) | (q[3])
Rgate(5.511) | (q[4])
Rgat

Everything has turned into beamsplitters and rotation gates! This is *definitely* not something you would want to do by hand, so compilers are a very valuable tool!

*Tip: You might have noticed there are some interesting patterns in which modes the beamsplitters are applied to. There are multiple ways to decompose an arbitrary unitary. The default method in Strawberry Fields is a rectangular mesh of beamsplitters, as detailed in the [docs for `Interferometer`](https://strawberryfields.readthedocs.io/en/stable/code/api/strawberryfields.ops.Interferometer.html).  Try switching to a triangular mesh and see how the pattern changes!*

Now, this kind of decomposition works in theory. But in practice, the hardware may not be set up in such as way as to implement these particular arrangements of beamsplitters. For example, Xanadu's X8 device performs a sequence of operations of a prescribed form:

<img src="X8.png" width=600>

We not only need general-purpose compilers, but we also need device-specific compilers to ensure we can run on our hardware.

**Exercise 4.** Design your own circuit to run on the X8 device! You'll need:

 - parameters for the two-mode squeezers ($r_0, r_1, r_2, r_3$) - these must be either 1.0 or 0.0
 - a 4x4 unitary operation (note that the same unitary is applied to both sets of modes)
 
Then, compile your program using the [`Xunitary`](https://strawberryfields.readthedocs.io/en/stable/code/api/strawberryfields.compilers.Xunitary.html) compiler. This will compile the program to a form compatible for the X class of chips, but without the need for device access (this we'll do in the next exercise, and you'll need an API key for that). Notice anything different from the compilation above? Which gates are being used?

In [6]:
prog = sf.Program(8, name="remote_job1")

U = sf.utils.random_interferometer(4)

with prog.context as q:
    # Initial squeezed states
    # Allowed values are r=1.0 or r=0.0
    ops.S2gate(1.0) | (q[0], q[4])
    ops.S2gate(1.0) | (q[1], q[5])
    ops.S2gate(1.0) | (q[3], q[7])

     # Interferometer on the signal modes (0-3)
    ops.Interferometer(U) | (q[0], q[1], q[2], q[3])
    ops.BSgate(0.543, 0.123) | (q[2], q[0])
    ops.Rgate(0.453) | q[1]
    ops.MZgate(0.65, -0.54) | (q[2], q[3])

    # *Same* interferometer on the idler modes (4-7)
    ops.Interferometer(U) | (q[4], q[5], q[6], q[7])
    ops.BSgate(0.543, 0.123) | (q[6], q[4])
    ops.Rgate(0.453) | q[5]
    ops.MZgate(0.65, -0.54) | (q[6], q[7])

    ops.MeasureFock() | q

In [7]:
prog2 = prog.compile(compiler="Xunitary")

In [8]:
prog2.print()

S2gate(0, 0) | (q[2], q[6])
S2gate(1, 0) | (q[0], q[4])
S2gate(1, 0) | (q[1], q[5])
S2gate(1, 0) | (q[3], q[7])
MZgate(0.9447, 4.983) | (q[0], q[1])
MZgate(0.9858, 5.029) | (q[2], q[3])
MZgate(0.8226, 1.091) | (q[1], q[2])
MZgate(1.577, 4.2) | (q[0], q[1])
MZgate(0.4071, 3.44) | (q[2], q[3])
MZgate(1.656, 4.463) | (q[1], q[2])
Rgate(5.457) | (q[0])
Rgate(2.368) | (q[1])
Rgate(5.603) | (q[2])
Rgate(0.04233) | (q[3])
MZgate(0.9447, 4.983) | (q[4], q[5])
MZgate(0.9858, 5.029) | (q[6], q[7])
MZgate(0.8226, 1.091) | (q[5], q[6])
MZgate(1.577, 4.2) | (q[4], q[5])
MZgate(0.4071, 3.44) | (q[6], q[7])
MZgate(1.656, 4.463) | (q[5], q[6])
Rgate(5.457) | (q[4])
Rgate(2.368) | (q[5])
Rgate(5.603) | (q[6])
Rgate(0.04233) | (q[7])
MeasureFock | (q[0], q[1], q[2], q[3], q[4], q[5], q[6], q[7])


**Exercise 5.** Now compile your algorithm for the X8, and run your algorithm on the hardware!

In [10]:
eng = sf.RemoteEngine("X8")
device = eng.device_spec
prog2 = prog.compile(device=device)

results = eng.run(prog2, shots=20)
print(results.samples)

2021-07-30 11:16:32,492 - INFO - The device spec X8_01 has been successfully retrieved.
2021-07-30 11:16:32,537 - INFO - Program previously compiled for X8_01 using Xunitary. Validating program against the Xstrict compiler.
2021-07-30 11:16:33,109 - INFO - Job 98367db4-f891-4456-adec-c0803f6264ef was successfully submitted.
2021-07-30 11:16:45,190 - ERROR - The remote job 98367db4-f891-4456-adec-c0803f6264ef failed due to an internal server error. Please try again. {'error-code': 'device-offline', 'error-detail': 'The requested hardware device is offline. Please try again during regular service hours.'}


FailedJobError: The remote job 98367db4-f891-4456-adec-c0803f6264ef failed due to an internal server error. Please try again. {'error-code': 'device-offline', 'error-detail': 'The requested hardware device is offline. Please try again during regular service hours.'}